In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load CIFAR-10 dataset
(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train[:80%]', 'train[80%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

# Preprocessing
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

BATCH_SIZE = 32

ds_train = ds_train.map(normalize_img).batch(BATCH_SIZE).prefetch(1)
ds_test = ds_test.map(normalize_img).batch(BATCH_SIZE).prefetch(1)


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, 3, activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train model
model.fit(ds_train, epochs=5, validation_data=ds_test)

test_loss, test_acc = model.evaluate(ds_test)
print(f"Test Accuracy: {test_acc:.4f}")


converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
with open('recycle_classifier.tflite', 'wb') as f:
    f.write(tflite_model)


Edge AI Prototype Report 

We trained a lightweight CNN model using the CIFAR-10 dataset (used as a proxy for recyclable items). The model achieved ~X% accuracy. We then converted it to a .tflite format using TensorFlow Lite, suitable for deployment on edge devices like a Raspberry Pi or mobile phone.

Edge AI Benefits:

Enables real-time processing without cloud latency.

Reduces bandwidth and energy consumption.

Operates in offline or low-connectivity environments (e.g., recycling bins in remote areas).

Deployment Steps:

Train and evaluate the model using TensorFlow.

Convert the model to .tflite.

Transfer the .tflite file to the edge device (e.g., Raspberry Pi).

Use TensorFlow Lite Interpreter to run inference on images captured by a local camera.